In [3]:
from qutip import *
print(qutip.__version__)
from IPython.display import Image


4.5.0


In [4]:
%matplotlib inline 

In [5]:
from numpy import pi

In [6]:
import copy
from qutip.qip.operations import *
from qutip.qip.circuit import * 
import numpy as np
from qutip.qip.device import Processor
from qutip.qip.device import CircularSpinChain, LinearSpinChain
from qutip.qip.noise import RandomNoise
from qutip.operators import sigmaz, sigmay, sigmax, destroy
from qutip.states import basis
from qutip.metrics import fidelity
from qutip.qip.operations import rx, ry, rz, hadamard_transform


In [7]:
q1 = QubitCircuit(4)
q1.add_gate("SNOT", [0])
q1.add_gate("SWAP", [2, 3])
q1.add_gate("SWAP", [1, 2])
q1.add_gate("SWAP", [0, 1])
q2 = QubitCircuit(9)
q2.add_gate("SNOT",[0])
q2.add_gate("SWAP",[0,3])



In [8]:
U_gate1 =  gate_sequence_product(q1.propagators())
U_gate2 =  gate_sequence_product(q2.propagators())
print(U_gate2.full().shape)
new_b = basis(16,0)
#np.matmul(U_gate, new_b)

(512, 512)


In [9]:
'''
tlist = np.linspace(0,2,20)
circuit_result = mesolve(U_gate, new_b,tlist)
circuit_result.states[-1] # the finial state
'''

'\ntlist = np.linspace(0,2,20)\ncircuit_result = mesolve(U_gate, new_b,tlist)\ncircuit_result.states[-1] # the finial state\n'

In [10]:
processor = Processor(N=4)
processor.add_control(swap(), targets=[0,1])
processor.add_control(swap(), targets=[2,3])
processor.add_control(swap(), targets=[2,1])
processor.set_all_tlist(np.array([0., pi/2., 2*pi/2, 3*pi/2]))
processor.pulses[0].coeff = np.array([1,1,1])

processor.pulses[1].coeff = np.array([0,1,1])

processor.pulses[2].coeff = np.array([0,0,1])

processor


In [11]:
p1 = CircularSpinChain(9)
p2 = CircularSpinChain(9)

U_list1 = p1.run(q1)
U_physical = gate_sequence_product(U_list1)

U_list2 = p2.run(q2)
U_physical2 = gate_sequence_product(U_list2)


In [12]:
(U_gate1 - U_physical).norm()
(U_gate2 - U_physical2).norm()

TypeError: Incompatible quantum object dimensions

In [13]:
basis0 = basis(2**9)
result = p1.run_state(basis0)
print(result.states[-1].tidyup(1.0e-4))
result2 = p2.run_state(basis0)
print(result2.states[-1].tidyup(1.0e-4))

Quantum object: dims = [[512], [1]], shape = (512, 1), type = ket
Qobj data =
[[0.000506  -0.70707752j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        

In [14]:
gassian_white = RandomNoise(rand_gen=np.random.normal, dt=0.1, loc=-0.05, scale=0.02)
processor_white = copy.deepcopy(p1)
processor_white.add_noise(gassian_white)  # gausian white noise

processor_white2 = copy.deepcopy(p2)
processor_white2.add_noise(RandomNoise(rand_gen=np.random.normal, dt=0.1, loc=-0.05, scale=0.02))  # gausian white noise

In [15]:
result_white = processor_white.run_state(basis0)
result_white.states[-1].tidyup(1.0e-4)

result_white2 = processor_white2.run_state(basis0)
result_white2.states[-1].tidyup(1.0e-4)

Quantum object: dims = [[512], [1]], shape = (512, 1), type = ket
Qobj data =
[[-0.02246201+0.00924899j]
 [-0.01713147-0.03338563j]
 [ 0.04317882-0.04132833j]
 [ 0.02704   +0.00118916j]
 [-0.0108888 +0.01661841j]
 [-0.02288686+0.05305737j]
 [-0.05764583+0.02183972j]
 [-0.01286779+0.01637269j]
 [ 0.02260075-0.06209409j]
 [-0.03957009-0.00474877j]
 [-0.05499173-0.01644098j]
 [-0.025303  -0.00118287j]
 [ 0.02238895-0.04500059j]
 [ 0.03745135-0.03291374j]
 [ 0.06164595-0.08059339j]
 [ 0.03106808-0.09483057j]
 [-0.02451874-0.02997671j]
 [-0.00896297+0.0333464j ]
 [-0.05470241+0.08441162j]
 [ 0.01453118-0.00875218j]
 [-0.06248399+0.03180192j]
 [ 0.04308338+0.02096348j]
 [ 0.00457079-0.00894377j]
 [ 0.03255506-0.09285642j]
 [-0.03798724+0.03680242j]
 [ 0.0435215 +0.07277601j]
 [-0.00709541+0.08883877j]
 [ 0.06085443-0.06829379j]
 [ 0.00644468+0.07589326j]
 [ 0.1097184 -0.00508506j]
 [ 0.06875489+0.03094501j]
 [ 0.03953377-0.05541638j]
 [ 0.00275062-0.03982129j]
 [-0.00267402-0.01332816j]
 [-0

In [16]:
fidelity(result.states[-1], result_white.states[-1])

0.013414523245128767

In [17]:
fidelity(result2.states[-1], result_white2.states[-1])

0.01601860815078354

In [18]:
N = 9
qc_Grover = QubitCircuit(9)


'''
qc_Grover.add_gate("SNOT",[0])
qc_Grover.add_gate("SNOT",[1])
qc_Grover.add_gate("SNOT",[2])
qc_Grover.add_gate("SNOT",[3])

qc_Grover.add_gate("CRZ",controls = [0], targets = [3],arg_value=pi/4)
'''
qc_Grover.add_gate("CNOT",controls = [0], targets = [2])
'''
qc_Grover.add_gate("CRZ",controls = [1], targets = [3],arg_value=-pi/4)
qc_Grover.add_gate("CNOT",controls = [0], targets = [1])
qc_Grover.add_gate("CRZ",controls = [1], targets = [3],arg_value=pi/4)
qc_Grover.add_gate("CNOT",controls = [1], targets = [2])
qc_Grover.add_gate("CRZ",controls = [2], targets = [3],arg_value=-pi/4)
qc_Grover.add_gate("CNOT",controls = [1], targets = [2])
qc_Grover.add_gate("CNOT",controls = [0], targets = [1])
qc_Grover.add_gate("CNOT",controls = [1], targets = [2])
qc_Grover.add_gate("CNOT",controls = [0], targets = [1])
qc_Grover.add_gate("CRZ",controls = [2], targets = [3],arg_value=pi/4)
qc_Grover.add_gate("CNOT",controls = [1], targets = [2])
qc_Grover.add_gate("CRZ",controls = [2], targets = [3],arg_value=-pi/4)
qc_Grover.add_gate("CNOT",controls = [0], targets = [2])
qc_Grover.add_gate("CRZ",controls = [2], targets = [3],arg_value=pi/4)
qc_Grover.add_gate("CRZ",controls = [2], targets = [3],arg_value=pi/4)
'''
qc_Grover2 = qc_Grover.adjacent_gates()






In [19]:
qc_Grover2.png

FileNotFoundError: [Errno 2] No such file or directory: 'qcirc.png'

In [20]:
gassian_white
for pulses in processor_white.get_noisy_pulses():
    pulses.print_info()

----------------------------------------------------------------------
The pulse contains: 1 coherent noise elements and 0 Lindblad noise elements.

Ideal pulse:
{'qobj': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]], 'targets': 0, 'tlist': array([ 0.   ,  0.125,  0.625,  0.75 ,  1.75 ,  3.   ,  3.5  ,  4.75 ,
        5.25 ,  6.5  ,  7.   ,  8.25 ,  8.75 , 10.   , 10.5  , 11.75 ,
       12.25 , 13.5  , 14.   , 15.25 , 15.75 , 17.   , 17.5  ]), 'coeff': array([ 0.        ,  1.57079633,  0.        ,  1.57079633,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -1.57079633,  0.        ,  0.        ,
        0.        , -1.57079633])}

Coherent noise:
{'qobj': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]], 'targets': 0, 'tlist'

In [21]:
print(len(p1.pulses))
print(len(processor_white.get_noisy_pulses()))

27
27


In [22]:
for pulse in processor_white2.pulses:
    pulse.print_info()

----------------------------------------------------------------------
The pulse contains: 0 coherent noise elements and 0 Lindblad noise elements.

Ideal pulse:
{'qobj': Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]], 'targets': 0, 'tlist': array([ 0.   ,  0.125,  0.625,  0.75 ,  1.75 ,  3.   ,  3.5  ,  4.75 ,
        5.25 ,  6.5  ,  7.   ,  8.25 ,  8.75 , 10.   , 10.5  , 11.75 ,
       12.25 , 13.5  , 14.   , 15.25 , 15.75 , 17.   , 17.5  , 18.75 ,
       19.25 , 20.5  , 21.   , 22.25 , 22.75 , 24.   , 24.5  , 25.75 ,
       26.25 , 27.5  , 28.   ]), 'coeff': array([ 0.        ,  1.57079633,  0.        ,  1.57079633,  0.        ,
       -1.57079633,  0.        ,  0.        ,  0.        , -1.57079633,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.

In [210]:
cnot() 

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]

In [24]:
a

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   0.5  0.5]
 [ 0.   0.   0.5 -0.5]]

In [25]:
a.inv()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  1. -1.]]

In [33]:
rx(pi/2)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[0.70710678+0.j         0.        -0.70710678j]
 [0.        -0.70710678j 0.70710678+0.j        ]]

In [203]:
def user_gate1(a):
    b= 1-a 
    mat = np.zeros((4,4), dtype = np.complex)
    mat[0,0] = mat[1,1] = 1
    mat[2,2] = b/np.sqrt(a**2 + b**2)
    mat[3,3] = -mat[2,2]
    mat[3,2] = mat[2,3] =  a/np.sqrt(a**2 +b**2)
    return Qobj(mat, dims=[[2, 2], [2, 2]])

In [204]:
cirq = QubitCircuit(4)
cirq.user_gates = {"NU":user_gate1}


In [205]:
cirq.add_gate("NU", controls = None, targets = [0,1], arg_value = 1)
cirq.add_gate("NU", controls = None, targets = [1,0], arg_value = 1)
cirq.add_gate("NU", controls = None, targets = [0,1], arg_value = 1)

cirq.add_gate("NU", controls = None, targets = [0,1], arg_value = 1)
cirq.add_gate("NU", controls = None, targets = [1,0], arg_value = 1)
cirq.add_gate("NU", controls = None, targets = [0,1], arg_value = 1)

In [206]:
t = cirq.propagators()

In [207]:
tt = gate_sequence_product(t)

In [208]:
tt.isunitary

True

In [209]:
tt

Quantum object: dims = [[2, 2, 2, 2], [2, 2, 2, 2]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]